<h1 style="text-align: center;">AAG - Méthodes Classiques en IA</h1>


## KPPV

### Préparation du dataset & création des corpus

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Chargement du dataset dans une DataFrame
dataset_R6 = pd.read_csv("./DatasetR6/S5_operators.csv", sep=';')

# on calcule le w/r
dataset_R6.insert(dataset_R6.shape[1],"w/r", (dataset_R6["nbwins"]/dataset_R6["nbpicks"]), True) 
dataset_R6.drop(columns=["nbwins","nbpicks","role"], inplace=True)

# on calcule le k/d
dataset_R6.loc[dataset_R6['nbdeaths'] == 0] = 1
# on a une ligne qui fait que des 1 partout 
dataset_R6.drop(axis=0, index=dataset_R6[dataset_R6['operator'] == 1].index, inplace=True)
dataset_R6['k/d'] = dataset_R6['nbkills'] / dataset_R6['nbdeaths']


# on merge et enleve les colonnes plateformes et dateid sans perdre les données associées
dataset_R6 = dataset_R6.groupby(['skillrank','operator', 'primaryweapon', 'secondaryweapon', 'secondarygadget'], as_index=False)[['w/r', 'k/d']].mean()

# discretisation de w/r
dataset_R6.insert(dataset_R6.shape[1],"target", (dataset_R6["w/r"]>=0.5).astype(int), True) 

dataset_R6.drop(columns=["w/r"], inplace=True)

#discrétisation de k/d
dataset_R6['k/d'] = (dataset_R6["k/d"]>=1.0).astype(int)


# changement de la taille d'affichage des dataframes
#pd.set_option('display.max_rows', dataset_R6.shape[0]+1)

corpus_annot = dataset_R6.copy()
corpus_annot = corpus_annot[['target']]

corpus_non_annot = dataset_R6.copy()
corpus_non_annot.drop(columns=["target"], inplace=True)

x_train, x_test, y_train, y_test = train_test_split(corpus_non_annot, corpus_annot, test_size=1/3, random_state=42, stratify=corpus_annot)

corpus_test_non_annot = x_test.copy()

corpus_train = x_train.copy()
corpus_train['target'] = y_train.copy()

corpus_verif = x_test.copy()
corpus_verif['target'] = y_test.copy()

#corpus_verif.head(30)


,skillrank,operator,primaryweapon,secondaryweapon,secondarygadget,k/d,target
583,Copper,SWAT-PULSE,M1014,M45 MEUSOC,NITRO CELL,0,0
962,Gold,GIGN-RESERVE,F2,LFP586,FRAG GRENADE,0,0
992,Gold,GIGN-ROOK,P90,P9,DEPLOYABLE SHIELD,0,1
1612,Silver,GIGN-ROOK,SG-CQB,P9,DEPLOYABLE SHIELD,0,1
368,Copper,GIGN-RESERVE,SG-CQB,LFP586,DEPLOYABLE SHIELD,0,0
211,Bronze,SPETSNAZ-FUZE,AK-12,PMM,STUN GRENADE,1,0
1199,Gold,SWAT-RESERVE,556XI,5.7 USG,STUN GRENADE,0,0
2122,Unranked,SWAT-RESERVE,556XI,5.7 USG,FRAG GRENADE,0,0
1997,Unranked,SAS-RESERVE,M590A1,P226 MK 25,FRAG GRENADE,0,0
43,Bronze,GIGN-MONTAGNE,Shield,P9,STUN GRENADE,0,0


In [79]:
'''
fonction qui calcule la distance de hamming entre deux vecteurs

'''

def hamming(x, y):
    cpt = 0
    #print(x, y)
    for i in range(0, len(y)-1):
        if x[i] != y[i]:
            cpt += 1
    return cpt


'''
fonction qui réalise l'alogrithme des k plus proches voisins pour un corpus de test

'''

def kppv_hyp_row(corpus_train,corpus_test, k):
    nouveau_corpus = corpus_test.copy()
    for row in corpus_test.itertuples():
        matrice_distance = []
        for row2 in corpus_train.itertuples():
            matrice_distance.append(hamming(row, row2))
                
        matrice_des_indices_des_elements_les_plus_proches = []
        #print(matrice_distance)

        for i in range(0, k):
            matrice_des_indices_des_elements_les_plus_proches.append(matrice_distance.index(min(matrice_distance)))
            matrice_distance[matrice_distance.index(min(matrice_distance))] = 1000

        cpt_0 = 0
        cpt_1 = 0
        #print(matrice_des_indices_des_elements_les_plus_proches)
        
        for i in matrice_des_indices_des_elements_les_plus_proches:
            #print(i,corpus_train.iloc[i]['target'])
            if corpus_train.iloc[i]['target'] == 1:
                cpt_1 += 1
            else:
                cpt_0 += 1
        
        if cpt_1 > cpt_0:
            nouveau_corpus.at[row.Index, 'target'] = 1
        else:
            nouveau_corpus.at[row.Index, 'target'] = 0

    return nouveau_corpus


In [80]:
corpus_train2 = corpus_train.head(10)
corpus_test = corpus_test_non_annot.head(30)



a = kppv_hyp_row(corpus_train,corpus_test_non_annot, 3)
print(a)

     skillrank          operator primaryweapon secondaryweapon  \
583     Copper        SWAT-PULSE         M1014      M45 MEUSOC   
962       Gold      GIGN-RESERVE            F2          LFP586   
992       Gold         GIGN-ROOK           P90              P9   
1612    Silver         GIGN-ROOK        SG-CQB              P9   
368     Copper      GIGN-RESERVE        SG-CQB          LFP586   
...        ...               ...           ...             ...   
1256  Platinum          GIGN-DOC           MP5              P9   
2146  Unranked     SWAT-THERMITE         M1014         5.7 USG   
2075  Unranked  SPETSNAZ-RESERVE       9x19VSN          GSH-18   
277     Bronze        SWAT-PULSE         UMP45         5.7 USG   
94      Bronze       GSG9-BANDIT           MP7             P12   

        secondarygadget  k/d  target  
583          NITRO CELL    0     0.0  
962        FRAG GRENADE    0     0.0  
992   DEPLOYABLE SHIELD    0     1.0  
1612  DEPLOYABLE SHIELD    0     1.0  
368   DEPLOY

In [81]:
nb_true, nb_false  = kppv_hyp_row(corpus_train,corpus_test_non_annot, 3).eq(corpus_verif)['target'].value_counts()
print(f'Nous avons {nb_true} bonnes annotations et {nb_false} mauvaises annotations sur {nb_true+nb_false} données ce qui donne un accuracy de {(nb_true/(nb_true+nb_false))*100}%')


Nous avons 618 bonnes annotations et 99 mauvaises annotations sur 717 données ce qui donne un accuracy de 86.19246861924687%
